# COGS 108 - Data Checkpoint

# Names

- Amogh Manjunath
- Govinda Sahoo
- Jonathan Ly
- Rohan Sreedhar

<a id='research_question'></a>
# Research Question

What metrics are correlated with passenger/customer satisfaction on airlines? How could different types of delays impact customer satisfaction?

# Dataset(s)

- **Name of dataset:** US Airline Passenger Satisfaction

- **Link to dataset:** https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction

- **Number of observations:** 129,880

This dataset comes from a 2015 survey on arriving passengers of full-service carriers at an airport, asking them to provide demographic data as well as answers on their loyalty, reason for travel, and arrival and departure delays. It also asked the passengers to rate their satisfaction with various features from 1-5, with 5 being the highest and 0 being "not applicable." This dataset was also used in the referenced article by Noviantoro and Huang.


- **Name of dataset:** 2015 Flight Delays and Cancellation

- **Link to dataset:** https://www.kaggle.com/datasets/usdot/flight-delays?select=flights.csv

- **Number of observations:** 5,819,079. However, only a subset of the dataset will be used.

This dataset is provided by the US Department of Transportation, and it contains information on every flight in the United States in the Year 2015. The information collected  includes arrival and departure delay times, as well as reasons for them.

- **Relation between datasets**

The first dataset has many observations to work with, and contains 20 metrics to compare. The second dataset, from the US Department of Transportation, has information on delays and the reasons for the delay. Analysis on the second dataset should reveal which reason for a delay causes the longest delays. Analysis on the first dataset may reveal how delays impact customer satisfaction. Putting them together can provide information on how different types of delays impact customer satisfaction.

# Setup

In [1]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [5]:
#install some modules
import sys  
!{sys.executable} -m pip install --user matplotlib
!{sys.executable} -m pip install --user openpyxl


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python3.9 -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


First, we import the necessary modules to deal with the data we are about to use. 
Then we import the datasets we will use.  

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

satData = pd.read_excel("satisfaction.xlsx")
delayData = pd.read_csv("delays.csv")

Let's look at the details of the 'Satisfaction' dataset:

In [8]:
satData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 129880 non-null  int64  
 1   satisfaction_v2                    129880 non-null  object 
 2   Gender                             129880 non-null  object 
 3   Customer Type                      129880 non-null  object 
 4   Age                                129880 non-null  int64  
 5   Type of Travel                     129880 non-null  object 
 6   Class                              129880 non-null  object 
 7   Flight Distance                    129880 non-null  int64  
 8   Seat comfort                       129880 non-null  int64  
 9   Departure/Arrival time convenient  129880 non-null  int64  
 10  Food and drink                     129880 non-null  int64  
 11  Gate location                      1298

We don't need the first column, 'id', since this column will have no correlation to what we are trying to discover. 

In [9]:
#drop first column
satData.drop(columns=satData.columns[0], axis=1, inplace=True)

A bunch of the columns contain categorical data, for example, given as integers 0 through 5. These are listed as 'int64' types, but we really want the data to be of the type 'category' 

In [10]:
toCategory = [0, 1, 2, 4, 5] + list(range(7, 21))
satData.iloc[:,toCategory] = satData.iloc[:,toCategory].astype('category')
satData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype   
---  ------                             --------------   -----   
 0   satisfaction_v2                    129880 non-null  category
 1   Gender                             129880 non-null  category
 2   Customer Type                      129880 non-null  category
 3   Age                                129880 non-null  int64   
 4   Type of Travel                     129880 non-null  category
 5   Class                              129880 non-null  category
 6   Flight Distance                    129880 non-null  int64   
 7   Seat comfort                       129880 non-null  category
 8   Departure/Arrival time convenient  129880 non-null  category
 9   Food and drink                     129880 non-null  category
 10  Gate location                      129880 non-null  category
 11  Inflight wifi service     

We've cleaned up the data to eliminate unnecessary columns and change any incorrect datatypes to more accurate ones. 
Now we look at the delayData info. 

In [17]:
delayData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 7 columns):
 #   Column               Dtype  
---  ------               -----  
 0   DEPARTURE_DELAY      float64
 1   ARRIVAL_DELAY        float64
 2   AIR_SYSTEM_DELAY     float64
 3   SECURITY_DELAY       float64
 4   AIRLINE_DELAY        float64
 5   LATE_AIRCRAFT_DELAY  float64
 6   WEATHER_DELAY        float64
dtypes: float64(7)
memory usage: 310.8 MB


We don't need most of these columns. We only need the ones pertaining to delays. 

In [16]:
delayData.drop(columns=delayData.columns[0:11], axis=1, inplace=True)
delayData.drop(columns=delayData.columns[1:11], axis=1, inplace=True)
delayData.drop(columns=delayData.columns[2:5], axis=1, inplace=True)

We also want to remove any observations where the departure delays and arrival delays are both negative or 0. This likely indicates that the flight departed and arrived early or on-time. 

We'll also delete observations where all the contributing factors for delays are NaN.

In [18]:
delayData = delayData.drop(delayData.index[(delayData['DEPARTURE_DELAY'] <= 0) & (delayData['ARRIVAL_DELAY'] <= 0)])

In [20]:
delayData = delayData.dropna(subset=['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2798869 entries, 2 to 5819078
Data columns (total 7 columns):
 #   Column               Dtype  
---  ------               -----  
 0   DEPARTURE_DELAY      float64
 1   ARRIVAL_DELAY        float64
 2   AIR_SYSTEM_DELAY     float64
 3   SECURITY_DELAY       float64
 4   AIRLINE_DELAY        float64
 5   LATE_AIRCRAFT_DELAY  float64
 6   WEATHER_DELAY        float64
dtypes: float64(7)
memory usage: 170.8 MB
